# 05. LoRAによるファインチューニング (概念デモ)

RAGがLLMに「外部知識」を与える技術だったのに対し、**ファインチューニング**はLLM自体の「振る舞い」や「応答スタイル」を特定のタスクに合わせて適応させる技術です。

しかし、巨大なLLM全体を再学習させるのは、膨大な計算資源（GPUメモリ、時間）が必要となり、非常にコストがかかります。

そこで登場するのが **LoRA (Low-Rank Adaptation)** という、**PEFT (Parameter-Efficient Fine-Tuning)** の一種です。
LoRAは、元のLLMの重みは固定したまま、新たに追加したごく一部の小さな重み（アダプタ）だけを学習します。これにより、少ない計算資源で効率的にファインチューニングを行うことができます。

このノートブックでは、**特定の出力形式（JSON）を遵守するようにLoRAで事前学習されたアダプタ**を読み込み、その効果を体験します。

## 事前準備

`00_setup_common.ipynb` を実行して、ライブラリのインストールと設定が完了していることを確認してください。

In [ ]:
# 編集禁止セル
# セットアップの確認と共通モジュールのインポート
import os
import sys
import torch
from google.colab import drive

if not os.path.isdir('/content/drive'): drive.mount('/content/drive')
repo_path = '/content/llm_lab'
if not os.path.exists(repo_path): print('リポジトリが見つかりません。')
else: os.chdir(repo_path)
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

try:
    from src.common import load_llm, generate_text
    from src.lora import load_lora_adapter
    from peft import PeftModel
    print('共通モジュールのインポートが完了しました。')
except ImportError as e:
    print(f'共通モジュールのインポートに失敗しました: {e}')

## LoRAアダプタの準備

この演習では、あらかじめ用意された学習済みLoRAアダプタを使用します。
このアダプタは、`data/lora/lora_train_sample.jsonl` のようなデータセット（質問と、特定のJSON形式の回答のペア）を使って学習された、という想定です。

（実際のアダプタは `06_lora_qlora_exercise.ipynb` で作成しますが、ここでは概念を理解するため、学習済みのものが既にあるとして進めます）

まず、ダミーの学習済みアダプタをHugging Face Hubからダウンロードします。
これは、`stabilityai/japanese-stablelm-3b-4e1t-instruct` に対して、JSON出力をするようにLoRAでチューニングされたモデルです。

In [ ]:
# 編集禁止セル
from src.lora import create_lora_model, train_lora

adapter_path = "./lora_adapters/demo_adapter"

# デモ用アダプタが存在しない場合、簡易学習を行って作成する
if not os.path.exists(adapter_path):
    print('デモ用アダプタを作成します（これには数分かかります）...')
    
    # 1. 学習用モデルの一時ロード
    try:
        tmp_model, tmp_tokenizer = load_llm(use_4bit=True)
        
        # 2. LoRAモデルの作成
        tmp_lora_model = create_lora_model(tmp_model)
        
        # 3. 簡易学習の実行 (data/lora/lora_train_sample.jsonl を使用)
        # デモ用なのでステップ数を少なく設定
        train_lora(
            model=tmp_lora_model,
            tokenizer=tmp_tokenizer,
            train_dataset_path='data/lora/lora_train_sample.jsonl',
            output_dir=adapter_path,
            max_steps=30,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            learning_rate=2e-4
        )
        
        # 4. パスの調整 (train_loraは output_dir/final_adapter に保存するため)
        if os.path.exists(os.path.join(adapter_path, "final_adapter")):
            adapter_path = os.path.join(adapter_path, "final_adapter")
            
        # 5. メモリ解放
        del tmp_model, tmp_lora_model, tmp_tokenizer
        torch.cuda.empty_cache()
        print('デモ用アダプタの作成が完了しました。')
        
    except Exception as e:
        print(f'アダプタ作成中にエラーが発生しました: {e}')
else:
    print('デモ用アダプタは既に存在します。')
    if os.path.exists(os.path.join(adapter_path, "final_adapter")):
        adapter_path = os.path.join(adapter_path, "final_adapter")

## ベースモデルのロード

LoRAアダプタを適用する前の、素のLLM（ベースモデル）をロードします。

In [ ]:
# 編集禁止セル
base_model, tokenizer = None, None
try:
    # LoRAアダプタを後から適用するため、ここでは `load_llm` を使わずに直接ロードします
    from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
    from src.common import DEFAULT_MODEL_ID

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    
    base_model = AutoModelForCausalLM.from_pretrained(
        DEFAULT_MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
    )
    tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL_ID)
    
    print("ベースモデルのロードが完了しました。")
except Exception as e:
    print(f'モデルのロード中にエラーが発生しました: {e}')

## LoRA適用前 vs 適用後 の比較

同じプロンプトを使って、LoRAアダプタを適用する前と後で、LLMの応答がどのように変わるかを見てみましょう。
プロンプトでは、応答をJSON形式にするように指示します。

In [ ]:
# 編集禁止セル
question = "『星屑のメモリー』の主人公について教えて。"
instruction = "回答は必ず以下のJSON形式で出力してください。\n{ \"answer\": \"...\", \"confidence\": \"high|medium|low\" }"

prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{question}\n{instruction}\n\n### 応答:\n"

### ケース1: LoRA適用前 (ベースモデル)

プロンプトでJSON形式を指示しても、必ずしもその通りに出力してくれるとは限りません。しばしば、形式が崩れたり、余計なテキストが付加されたりします。

In [ ]:
# 編集禁止セル
if base_model and tokenizer:
    print("--- LoRA適用前の回答 ---")
    generated_text = generate_text(base_model, tokenizer, prompt, max_new_tokens=128)
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

### ケース2: LoRA適用後

次に、ベースモデルに学習済みLoRAアダプタを適用します。
これにより、モデルはJSON形式で出力する「振る舞い」を学習しているため、指示に忠実に従うようになります。

In [ ]:
# 編集禁止セル
lora_model = None
try:
    # ベースモデルにLoRAアダプタをロード
    lora_model = PeftModel.from_pretrained(base_model, adapter_path)
    lora_model.eval()
    print("LoRAアダプタの適用が完了しました。")
except Exception as e:
    print(f'LoRAアダプタの適用中にエラーが発生しました: {e}')

In [ ]:
# 編集禁止セル
if lora_model and tokenizer:
    print("--- LoRA適用後の回答 ---")
    generated_text = generate_text(lora_model, tokenizer, prompt, max_new_tokens=128)
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("LoRAモデルが準備できていません。")

## まとめ

このノートブックでは、LoRAによるファインチューニングの概念とその効果を学びました。

- **LoRAの効果**: 特定のタスク（今回はJSON形式での出力）に合わせてLLMの振る舞いを変化させることができる。
- **RAGとの違い**: RAGが「知識」を外部から与えるのに対し、LoRAはLLM自体の「スキル」や「性格」を変化させるイメージ。

LoRAは、応答スタイルの統一、特定の専門タスクへの特化、不適切な表現の抑制など、様々な目的に利用できます。

今回は学習済みのアダプタを使いましたが、次の `06_lora_qlora_exercise.ipynb` では、いよいよ自分たちの手でQLoRA（4bit量子化モデルに対するLoRA）の学習を行います。

### メモリ解放

In [ ]:
# 編集禁止セル
import gc
if 'base_model' in locals() and base_model is not None: del base_model
if 'lora_model' in locals() and lora_model is not None: del lora_model
if 'tokenizer' in locals() and tokenizer is not None: del tokenizer
base_model, lora_model, tokenizer = None, None, None
gc.collect()
torch.cuda.empty_cache()
print("モデルを解放し、GPUキャッシュをクリアしました。")